In [1]:
#!pip install beautifulsoup4 requests

In [2]:

import requests
from bs4 import BeautifulSoup

In [3]:

url = 'https://Your-url'
headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

r = requests.get("https://www.materialepyramiden.dk/",headers=headers,timeout=10)
soup = BeautifulSoup(r.content)
print(soup)

<!DOCTYPE html>
<html lang="da_DA" xml:lang="da" xmlns="http://www.w3.org/1999/xhtml">
<head>
<meta content="sustainability, construction materials, pyramid, pyramide, EPD, livscyklus, lifecycle, analysis, architecture, construction, byggeriet, bygninger, tool, værktøj, digital, materialer, bæredygtighed, livscyklusanalyse, LCA, KADK, Cinark, Vandkunsten" name="keywords"/>
<meta content="KADK/CINARK" name="Copyright"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="width=device-width, initial-scale=0.86, minimal-ui" id="viewport" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black" name="apple-mobile-web-app-status-bar-style"/>
<script charset="UTF-8" src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.j

In [4]:

results = soup.find_all("div", class_="mat-overlay")
links =[]
for i in range(len(results)):
    links.append(results[i].attrs['data-link'])

print(links)
print(len(links))

['https://materialepyramiden.dk/?materialer=fyr-gran-beklaedningsbraedder', 'https://materialepyramiden.dk/?materialer=eg-ask-ceder-boeg-laerk-beklaedningsbraedder', 'https://materialepyramiden.dk/?materialer=traefiberplade-haard-masonit-mdf-hdf', 'https://materialepyramiden.dk/?materialer=traefiberplade-bloed-undertag-og-vindplade', 'https://materialepyramiden.dk/?materialer=spaanplade', 'https://materialepyramiden.dk/?materialer=plankegulv-eg-fyr', 'https://materialepyramiden.dk/?materialer=osb', 'https://materialepyramiden.dk/?materialer=modificeret-trae-varmebehandlet', 'https://materialepyramiden.dk/?materialer=modificeret-trae-vaeskebehandlet', 'https://materialepyramiden.dk/?materialer=lamelgulv-eg-boeg-ask', 'https://materialepyramiden.dk/?materialer=kork', 'https://materialepyramiden.dk/?materialer=hampekalk', 'https://materialepyramiden.dk/?materialer=graesisolering', 'https://materialepyramiden.dk/?materialer=pandeplade', 'https://materialepyramiden.dk/?materialer=maling-mat

In [6]:
# DOWNLOAD ALL MATERIAL FILES

for i in range(len(links)):
    name = links[i].split("=")[-1]
    

    url = links[i]
    print(f"downloading {name} from {url}")
    ua = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    response  = requests.get(url, headers={"User-Agent": ua})
    if response.status_code == 200:
        with open(f"./materialer/{name}.html","wb") as file:
            file.write(response.content)
            file.close()
    else:
        print("Bad response code :", response.status_code, url)
    

print("Done")

downloading fyr-gran-beklaedningsbraedder from https://materialepyramiden.dk/?materialer=fyr-gran-beklaedningsbraedder
downloading eg-ask-ceder-boeg-laerk-beklaedningsbraedder from https://materialepyramiden.dk/?materialer=eg-ask-ceder-boeg-laerk-beklaedningsbraedder
downloading traefiberplade-haard-masonit-mdf-hdf from https://materialepyramiden.dk/?materialer=traefiberplade-haard-masonit-mdf-hdf
downloading traefiberplade-bloed-undertag-og-vindplade from https://materialepyramiden.dk/?materialer=traefiberplade-bloed-undertag-og-vindplade
downloading spaanplade from https://materialepyramiden.dk/?materialer=spaanplade
downloading plankegulv-eg-fyr from https://materialepyramiden.dk/?materialer=plankegulv-eg-fyr
downloading osb from https://materialepyramiden.dk/?materialer=osb
downloading modificeret-trae-varmebehandlet from https://materialepyramiden.dk/?materialer=modificeret-trae-varmebehandlet
downloading modificeret-trae-vaeskebehandlet from https://materialepyramiden.dk/?materia

In [8]:
# import OS module
import os
from tkinter.tix import DirList
import sqlite3
 

In [ ]:


# Get the list of all files and directories
path = "./materialer/"
dir_list = os.listdir(path)
sqliteConnection = sqlite3.connect('./dataBase/materialDB_20250129.db')

cursor = sqliteConnection.cursor()
tableName = "materials"

query = f"DROP TABLE {tableName}"
cursor.execute(query)


# query = f"""CREATE TABLE IF NOT EXISTS {tableName}(
#     name	TEXT NOT NULL UNIQUE,
#     category TEXT NOT NULL,
#     density NUMERIC NOT NULL,
#     A1toA3 NUMERIC NOT NULL,
#     ODB NUMERIC NOT NULL,
#     POCP NUMERIC NOT NULL,
#     EP NUMERIC NOT NULL,
#     AP NUMERIC NOT NULL,
# 	PRIMARY KEY("name")
# );"""
query = f"""CREATE TABLE IF NOT EXISTS {tableName}(
    "name"	TEXT NOT NULL UNIQUE,
	"category"	TEXT NOT NULL,
	"density"	NUMERIC NOT NULL,
	"insulation"	NUMERIC,
	"description"	TEXT,
	"A1toA3"	NUMERIC NOT NULL,
	"ODP"	NUMERIC NOT NULL,
	"POCP"	NUMERIC NOT NULL,
	"EP"	NUMERIC NOT NULL,
	"AP"	NUMERIC NOT NULL,
	"C1toC4"	NUMERIC,
	"DataSource"	TEXT,
	"Notes"	TEXT,
	PRIMARY KEY("name")
);"""
cursor.execute(query)


cloumnList = "name,category,density,A1toA3,ODP,POCP,EP,AP"

#for i in range(len(dir_list)):
runLen = len(dir_list)
#runLen = 1
for i in range(runLen):
    filepath = f"{path}{dir_list[i]}"
    print(f"opening {filepath}")
    with open(filepath, "r",encoding='utf8') as file:
        soup = BeautifulSoup(file, "html")

        name = soup.find( class_ = "single-title single-text c1" ).find(class_ = "en").text
        print(f"found name {name}")

        density = ""
        insulation = ""
        cat = ""
        A1toA3 = ""
        ODP = ""
        POCP  = ""
        EP = ""
        AP = ""



        #table = soup.find('table',class_ = "single-table")
        rows = soup.find_all('tr', class_ = 'tr-line')
        print(len(rows))

        
        for row in rows:
            #print(row)
            #print("   ")
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Density:"):
                    density = row.find(class_ = 't-text t-number t4').text.split(" ")[0]
            
            if(row.find(class_ = 'en')):
                if(row.find(class_ = 'en').text == "Materialgroup:"):
                    cat = row.find(class_ = 't-text t-number t4').text
            
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Global Warming Potential [GWP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        A1toA3 = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Ozone Depletion Potential [ODP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        ODP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Photochemical Ozone Depletion Potential [POCP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        POCP = row.find(class_ = 't-text t-number t2').text

            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Eutrophication Potential [EP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        EP = row.find(class_ = 't-text t-number t2').text
       
            if(row.find(class_ = 't-text t1')):
                if(row.find(class_ = 't-text t1').text == "Acidification Potential [AP]"):
                    if(row.find(class_ = 't-text t3').text.split("/")[-1] == "m3"):
                        AP = row.find(class_ = 't-text t-number t2').text



            #print("   ")


        print(f"name:{name} , category:{cat}, density:{density}, A1toA3:{A1toA3}, ODP:{ODP},POCP:{POCP},EP:{EP},AP:{AP}" )
        
        query = f"""INSERT OR REPLACE INTO {tableName} ({cloumnList})
                    VALUES ('{name}' , '{cat}' , {density} , '{A1toA3}','{ODP}','{POCP}','{EP}','{AP}');"""
        
        print(query)
        cursor.execute(query)


        

        

      
result = cursor.fetchall()
print('SQLite Version is {}'.format(result))



sqliteConnection.commit()
cursor.close()

opening ./materialer/aalegraes-loesfyld.html
found name Eelgras / zostera
24
name:Eelgras / zostera , category:biobaseret, density:137.00, A1toA3:-4.99E+1, ODP:7.47E+0,POCP:1.36E+2,EP:2.58E-1,AP:3.49E+2
INSERT OR REPLACE INTO materials (name,category,density,A1toA3,ODP,POCP,EP,AP)
                    VALUES ('Eelgras / zostera' , 'biobaseret' , 137.00 , '-4.99E+1','7.47E+0','1.36E+2','2.58E-1','3.49E+2');
opening ./materialer/aluminum-plade.html
found name Aluminium sheet
24
name:Aluminium sheet , category:metal, density:2700.00, A1toA3:2.89E+4, ODP:5.58E+1,POCP:5.32E+4,EP:1.85E+1,AP:9.71E+4
INSERT OR REPLACE INTO materials (name,category,density,A1toA3,ODP,POCP,EP,AP)
                    VALUES ('Aluminium sheet' , 'metal' , 2700.00 , '2.89E+4','5.58E+1','5.32E+4','1.85E+1','9.71E+4');
opening ./materialer/aluvindue.html
found name Aluminium frame window
24
name:Aluminium frame window , category:komponenter, density:422.02, A1toA3:2.28E+3, ODP:8.93E+2,POCP:5.60E+3,EP:9.79E+0,AP:1.15E+